# EDB: parameterized design

This example shows how to
1. Set up an HFSS project using SimulationConfiguration class.
2. Create automatically parametrized design.

This image shows the layout created in this example:

<img src="_static\parameterized_design.png" width="600">


## Perform imports

Perform required imports.

In [1]:
import tempfile

import ansys.aedt.core
import pyedb
from pyedb.misc.downloads import download_file

## Create an instance of a pyedb.Edb object.

In [2]:
temp_dir = tempfile.TemporaryDirectory(suffix=".ansys")
target_aedb = download_file("edb/ANSYS-HSD_V1.aedb", destination=temp_dir.name)
print("Project is located in ", target_aedb)

# Select EDB version (change it manually if needed, e.g. "2025.1")
edb_version = "2025.2"
print(f"EDB version: {edb_version}")

edb = pyedb.Edb(edbpath=target_aedb, edbversion=edb_version)
print("AEDB file is located in {}".format(target_aedb))

Project is located in  C:\Users\ansys\AppData\Local\Temp\tmp0rnbp6vf.ansys\edb/ANSYS-HSD_V1.aedb
EDB version: 2025.2


C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\misc\decorators.py:55: UserWarning: Argument `edbversion` is deprecated for method `Edb`; use `version` instead.
  warnings.warn(
C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: Star initializing Edb 10:35:42.298260


PyEDB INFO: Edb version 2025.2


PyEDB INFO: Logger is initialized. Log file is saved to C:\Users\ansys\AppData\Local\Temp\pyedb_ansys.log.


...

PyEDB INFO: Cell main Opened


PyEDB INFO: Builder was initialized.


PyEDB INFO: open_edb completed in 9.0137 seconds.


PyEDB INFO: EDB initialization completed in 9.0777 seconds.


AEDB file is located in C:\Users\ansys\AppData\Local\Temp\tmp0rnbp6vf.ansys\edb/ANSYS-HSD_V1.aedb


## Prepare the layout for the simulation

The ``new_simulation_configuration()`` method creates an instance of
the ``SimulationConfiguration`` class. This class helps define all preprocessing steps
required to set up the PCB for simulation. After the simulation configuration has been defined,
they are applied to the EDB using the ``Edb.build_simulation()`` method.

In [3]:
simulation_configuration = edb.new_simulation_configuration()
simulation_configuration.signal_nets = [
    "PCIe_Gen4_RX0_P",
    "PCIe_Gen4_RX0_N",
    "PCIe_Gen4_RX1_P",
    "PCIe_Gen4_RX1_N",
]
simulation_configuration.power_nets = ["GND"]
simulation_configuration.components = ["X1", "U1"]
simulation_configuration.do_cutout_subdesign = True
simulation_configuration.start_freq = "OGHz"
simulation_configuration.stop_freq = "20GHz"
simulation_configuration.step_freq = "10MHz"

Now apply the simulation setup to the EDB.

In [4]:
edb.build_simulation_project(simulation_configuration)

PyEDB INFO: Building simulation project.


PyEDB INFO: Cutting out using method: 1


PyEDB INFO: Cutting out using method: 1


PyEDB INFO: -----------------------------------------


PyEDB INFO: Trying cutout with (0.001)*(1000.0)mm expansion size


...

PyEDB INFO: Padstack definition with zero plating ratio, defaulting to 20%


PyEDB INFO: Padstack definition with zero plating ratio, defaulting to 20%


PyEDB INFO: Padstack definition with zero plating ratio, defaulting to 20%


PyEDB INFO: Save Edb file completed in 0.0508 seconds.


True

## Parameterize

The layout can automatically be set up to enable parametric studies. For example, the
impact of antipad diameter or trace width on signal integrity performance may be invested
parametrically.

In [5]:
edb.auto_parametrize_design(layers=True, materials=True, via_holes=True, pads=True, antipads=True, traces=True)
edb.save_edb()
edb.close_edb()

PyEDB INFO: Save Edb file completed in 0.0475 seconds.


PyEDB INFO: Close Edb file completed in 0.0789 seconds.


True

## Open project in AEDT

All manipulations thus far have been executed using the EDB API, which provides fast,
streamlined processing of layout data in non-graphical mode. The layout and simulation
setup can be visualized by opening it using the 3D Layout editor in AEDT.

Note that there may be some delay while AEDT is being launched.

In [6]:
hfss = ansys.aedt.core.Hfss3dLayout(
    project=target_aedb,
    version=edb_version,
    non_graphical=False,
    new_desktop=True,
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_266e97f8-13f6-4a34-8ec4-613268f141a7.log is enabled.


...

PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: EDB folder C:\Users\ansys\AppData\Local\Temp\tmp0rnbp6vf.ansys\edb/ANSYS-HSD_V1.aedb has been imported to project ANSYS-HSD_V1


PyAEDT INFO: Active Design set to 0;main


PyAEDT INFO: Active Design set to 0;main


PyAEDT INFO: Aedt Objects correctly read


The following cell can be used to ensure that the design is valid for simulation.

In [7]:
validation_info = hfss.validate_full_design()
is_ready_to_simulate = True

PyAEDT INFO: #### Design Validation Checks###


PyAEDT INFO: **** Validation Completed Correctly


PyAEDT INFO: Ports Requested: None


PyAEDT INFO: Ports Defined: 8


PyAEDT INFO: Excitation name: U1.AN26.PCIe_Gen4_RX0_N


...

PyAEDT INFO: Excitation name: U1.AR25.PCIe_Gen4_RX1_P


PyAEDT INFO: Excitation name: X1.A2.PCIe_Gen4_RX0_P


PyAEDT INFO: Excitation name: X1.A3.PCIe_Gen4_RX0_N


PyAEDT INFO: Excitation name: X1.A5.PCIe_Gen4_RX1_P


PyAEDT INFO: Excitation name: X1.A6.PCIe_Gen4_RX1_N


In [8]:
for s in validation_info[0]:
    if "error" in s:
        print(s)
        is_ready_to_simulate = False

if is_ready_to_simulate:
    print("The model is ready for simulation.")
else:
    print("There are errors in the model that must be fixed.")

The model is ready for simulation.


## Release the application from the Python kernel

It is important to release the application from the Python kernel after
execution of the script. The default behavior of the ``release_desktop()`` method closes all open
projects and closes the application.

If you want to continue working on the project in graphical mode
after script execution, call the following method with both arguments set to ``False``.

In [9]:
hfss.release_desktop(close_projects=True, close_desktop=True)
temp_dir.cleanup()  # Remove the temporary folder and files. All data will be removd!

PyAEDT INFO: Desktop has been released and closed.
